In [10]:
import pandas as pd, pathlib, csv

MODEL_DIR = pathlib.Path("model")
OUT_CSV   = pathlib.Path("results_master.csv")
HEADER    = ["dataset","model","loss","ls","mixup","augmix",
             "loss_val", "accuracy","ece","adaece","classece","brier"]

if not OUT_CSV.exists():
    with open(OUT_CSV, "w", newline="") as f:
        csv.writer(f).writerow(HEADER)

rows = []

for hist in MODEL_DIR.glob("*/history.csv"):
    tag = hist.parent.name
    parts = tag.split("_")

    aug_part  = parts[-1]                 # 'aug' | 'noaug'
    mix_part  = parts[-2]                 # 'mx0.2'
    ls_part   = parts[-3]                 # 'ls0.1'

    dataset, model_name, *loss_parts = parts[:-3]
    loss = "_".join(loss_parts)           # сохраняем 'bsce_gra' целиком

    try:
        ls_val  = float(ls_part[2:])      # оборвали 'ls'
        mix_val = float(mix_part[2:])     # оборвали 'mx'
    except ValueError:
        print("⚠️  тег не парсится:", tag)
        continue

    aug_flag = aug_part == "aug"

    df_h = pd.read_csv(hist)
    best = df_h.loc[df_h["val_loss"].idxmin()]

    rows.append([dataset, model_name, loss, ls_val, mix_val, aug_flag,
                 round(best["val_loss"],4), round(best["val_acc"],4), round(best["val_ece"],4),
                 "", "", ""])

# добавляем без дубликатов
existing = pd.read_csv(OUT_CSV)
combined = pd.concat([existing, pd.DataFrame(rows, columns=HEADER)])
combined = combined.drop_duplicates(subset=HEADER[:6], keep="first")
combined.to_csv(OUT_CSV, index=False)

print(f"Добавлено {len(rows)} строк. {combined.shape[0]} итоговых записей")


Добавлено 44 строк. 44 итоговых записей


In [11]:
base   = pathlib.Path('model')
res    = pd.read_csv('results_master.csv')
calib  = pd.read_csv('calib_master.csv')

res = res.rename({'val_ece':'ece', 'val_adaece':'adaece',
                  'val_classece':'classece', 'val_brier':'brier'})

# объединяем "до" и "после"
df = res.merge(calib, how='left',
               on=['dataset','model','loss','ls','mixup','augmix'],
               suffixes=('_raw','_post'))


In [12]:
df

,dataset,model,loss,ls,mixup,augmix,accuracy_raw,ece_raw,adaece_raw,classece_raw,brier_raw,loss_val,method,accuracy_post,ece_post,adaece_post,classece_post,brier_post
0,cifar10,resnet50,dual_focal,0.0,0.2,False,10.24,0.0024,NaN,NaN,NaN,2.6418,NaN,NaN,NaN,NaN,NaN,NaN
1,cifar100,resnet50,bsce_gra,0.0,0.0,False,41.26,0.0450,NaN,NaN,NaN,2.0968,NaN,NaN,NaN,NaN,NaN,NaN
2,cifar10,resnet50,bsce_gra,0.0,0.0,False,72.16,0.1669,NaN,NaN,NaN,0.6331,NaN,NaN,NaN,NaN,NaN,NaN
3,cifar10,resnet50,dual_focal,0.1,0.0,False,67.56,0.1916,NaN,NaN,NaN,1.1530,NaN,NaN,NaN,NaN,NaN,NaN
4,cifar10,resnet50,cross_entropy,0.1,0.0,False,75.28,0.0633,NaN,NaN,NaN,0.0549,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,otto,mlp,dual_focal,0.1,0.0,False,ERROR,NaN,NaN,NaN,NaN,NaN,platt,0.778927,0.026016,0.019754,0.642620,0.292552
58,otto,mlp,dual_focal,0.1,0.0,False,ERROR,NaN,NaN,NaN,NaN,NaN,isotonic,0.766322,0.033685,0.034846,0.536612,0.304218
59,otto,mlp,dual_focal,0.1,0.0,False,ERROR,NaN,NaN,NaN,NaN,NaN,bbq,0.446025,0.020842,0.037965,0.465559,0.683887
60,otto,mlp,dual_focal,0.0,0.2,False,ERROR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
res

,dataset,model,loss,ls,mixup,augmix,accuracy,ece,adaece,classece,brier,loss_val
0,cifar10,resnet50,dual_focal,0.0,0.0,False,10.5200,0.002945,NaN,NaN,NaN,NaN
1,otto,mlp,focal,0.0,0.0,False,77.2786,0.085170,NaN,NaN,NaN,NaN
2,cifar10,resnet50,focal,0.0,0.0,False,22.1400,0.016362,NaN,NaN,NaN,NaN
3,otto,mlp,dual_focal,0.0,0.2,False,70.0711,0.154162,NaN,NaN,NaN,NaN
4,otto,mlp,bsce_gra,0.0,0.0,True,74.8546,0.136422,NaN,NaN,NaN,NaN
5,cifar10,resnet50,dual_focal,0.0,0.0,True,11.2400,0.008888,NaN,NaN,NaN,NaN
6,cifar100,resnet50,focal,0.0,0.2,False,5.2600,0.006838,NaN,NaN,NaN,NaN
7,cifar10,resnet50,cross_entropy,0.0,0.0,False,71.5000,0.011212,NaN,NaN,NaN,NaN
8,otto,mlp,focal,0.1,0.0,False,77.1493,0.079844,NaN,NaN,NaN,NaN
9,cifar10,resnet50,cross_entropy,0.1,0.0,False,56.5400,0.021104,NaN,NaN,NaN,NaN
